In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Two_Patterns")
bs = 100
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = bs


Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.


In [3]:
dtcr_model = DTCRModel(config)
dtcr_model

DTCRModel(
  (_encoder): BidirectionalDRNN(
    (_regular_drnn): DRNN(
      (cells): Sequential(
        (0): GRU(1, 100)
        (1): GRU(100, 50)
        (2): GRU(50, 50)
      )
    )
    (_backwards_drnn): DRNN(
      (cells): Sequential(
        (0): GRU(1, 100)
        (1): GRU(100, 50)
        (2): GRU(50, 50)
      )
    )
  )
  (_decoder): DTCRDecoder(
    (_rnn): GRU(400, 400, batch_first=True)
    (_linear): Linear(in_features=400, out_features=1, bias=True)
  )
)

In [4]:
mse = torch.nn.MSELoss()
opt = torch.optim.Adam(list(dtcr_model.parameters()), eps=5e-3)
print_interval = 5
for epoch in range(50):
    running_loss = 0.0
    for index, (sample_data, sample_label) in enumerate(train_dl):
        opt.zero_grad()

        inputs, latent_representation, reconstructed_inputs = dtcr_model(sample_data)
        reconstruction_loss = mse(reconstructed_inputs, sample_data)
        reconstruction_loss.backward()
        opt.step()

        running_loss += reconstruction_loss.item()
        if index % print_interval == print_interval - 1:
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, index + 1, running_loss / print_interval))
            print("Real values:")
            print(",".join(["{:.2f}".format(float(item)) for item in inputs[0]][:15]))
            print("Latent representation:")
            print(",".join(["{:.2f}".format(float(item)) for item in latent_representation[0]][:15]))
            print("Predicted:")
            print(",".join(["{:.2f}".format(float(item)) for item in reconstructed_inputs[0]][:15]))
            
            running_loss = 0.0


[1,     5] loss: 0.993
Real values:
-0.14,-0.47,-0.53,-0.47,-0.51,0.24,0.17,-0.23,0.24,0.13,-0.09,0.11,-0.02,0.34,-0.06
Latent representation:
0.06,-0.05,-0.06,0.02,-0.00,0.07,0.15,0.01,-0.14,0.12,-0.09,-0.04,-0.05,0.02,-0.07
Predicted:
0.01,0.01,0.02,0.01,0.01,0.01,-0.00,-0.01,-0.01,-0.01,-0.02,-0.02,-0.02,-0.02,-0.02
[1,    10] loss: 0.993
Real values:
0.12,0.47,0.29,-0.10,0.15,0.35,0.35,-0.16,0.13,-1.58,-1.58,-1.58,-1.58,-1.58,-1.58
Latent representation:
0.06,-0.05,-0.06,0.02,-0.00,0.07,0.15,0.01,-0.14,0.12,-0.09,-0.04,-0.04,0.02,-0.07
Predicted:
0.01,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.00,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01
[2,     5] loss: 0.992
Real values:
0.35,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,-1.58,-1.58,-1.58,-1.58,-1.58
Latent representation:
0.08,-0.05,-0.04,0.01,0.02,0.08,0.17,0.03,-0.11,0.09,-0.08,-0.01,-0.06,0.04,-0.09
Predicted:
0.02,0.03,0.04,0.04,0.04,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.01,0.01,0.01
[2,    10] loss: 0.992
Real values:
0.02,0.33,0.78,0.03